In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
from utils import CB_SAFE_PALLETE

warnings.simplefilter("ignore")

<div class="main-title">
<h1>SRAI</h1>
<p>Design and usage<p>
</div>

## What is SRAI?

<div class="center-content">
    <h3 style="margin: auto;">A toolbox for geospatial AI</h3>
</div>

<div class="center-content">
    <p style="margin: auto;">...that aims to standardize the domain</p>
</div>

<div>
    <img src="assets/SRAI-diagram.png" class="srai-diagram"/>
</div>

Specify the city

In [ ]:
CITY = "Basel"
COUNTRY = "Switzerland"

area_name = f"{CITY}, {COUNTRY}"
area_name

Download the area

In [ ]:
from srai.regionalizers import geocode_to_region_gdf

area = geocode_to_region_gdf(area_name)
area.explore(height=500)

## Loaders

* used to load spatial data from different sources
* unify loading into a single interface
* prepare data for the embedding methods

[API](https://kraina-ai.github.io/srai/latest/api/loaders/)<br>
[Examples](https://kraina-ai.github.io/srai/latest/examples/loaders/)

Types of loaders:

* GTFS
* OSM Online
* OSM Pbf
* OSM Way
* OSM Tile

### Let's create a city poster

In [ ]:
from srai.loaders.osm_loaders.filters import BASE_OSM_GROUPS_FILTER

BASE_OSM_GROUPS_FILTER

In [ ]:
from srai.loaders import OSMPbfLoader

features = (
    OSMPbfLoader()
    .load(area, {"highway": True, "water": True, "waterway": True})
    .clip(area)
)
features.head(3)

In [ ]:
features.explore()

Plot the poster

In [ ]:
import matplotlib.pyplot as plt
from utils import plot_poster

ax = plot_poster(features, CITY, COUNTRY)

plt.savefig("poster.png", facecolor="#ecedea", dpi=300)
plt.close()

<img src="poster.png" style="height: 900px; margin: auto" /> 

### GTFS (General Transit Feed Specification)

In [ ]:
from pathlib import Path
from srai.loaders import GTFSLoader
from srai.plotting import plot_regions

In [ ]:
area_vienna = geocode_to_region_gdf("Vienna, Austria")  
gtfs_file = Path("./data") / "vienna_gtfs.zip"  # from https://database.mobilitydata.org/

features = GTFSLoader().load(gtfs_file, skip_validation=True).clip(area_vienna)
features.head(3)

In [ ]:
features[["trips_at_16", "geometry"]].explore("trips_at_16", cmap="winter_r", tiles="CartoDB positron")

<div>
    <img src="assets/SRAI-diagram.png" class="srai-diagram"/>
</div>

## Regionalizers

* unify methods for dividing a given area into smaller regions. 
* can be based on spatial indexes.

[API](https://kraina-ai.github.io/srai/latest/api/regionalizers/)<br>
[Examples](https://kraina-ai.github.io/srai/latest/examples/regionalizers/)

Types of regionalizers:

* H3
* S2
* Voronoi
* Administative Boundary

### Spatial index regionalization

In [ ]:
from srai.regionalizers import H3Regionalizer
from srai.plotting import plot_regions

In [ ]:
regions = H3Regionalizer(resolution=9).transform(area)
plot_regions(regions, colormap=CB_SAFE_PALLETE)

### Data driven regionalization

In [ ]:
from srai.loaders import OSMOnlineLoader

In [ ]:
stops = OSMOnlineLoader().load(area, {"public_transport": "stop_position"})
stops.explore()

In [ ]:
from srai.regionalizers import VoronoiRegionalizer
warnings.simplefilter("ignore")

regionalizer = VoronoiRegionalizer(seeds=stops)
regions = regionalizer.transform(area)

plot_regions(regions, colormap=CB_SAFE_PALLETE)

### Administrative boundaries

In [ ]:
from srai.regionalizers import AdministrativeBoundaryRegionalizer 

In [ ]:
regionalizer = AdministrativeBoundaryRegionalizer(admin_level=9)
regions = regionalizer.transform(area)
plot_regions(regions, colormap=CB_SAFE_PALLETE)

<div>
    <img src="assets/SRAI-diagram.png" class="srai-diagram"/>
</div>

## Embedders

Unify methods for mapping regions into a vector space.

[API](https://kraina-ai.github.io/srai/latest/api/embedders/)<br>
[Examples](https://kraina-ai.github.io/srai/latest/examples/embedders/)

Types of embedders:

* Count
* Contextual Count
* GTFS2Vec
* Hex2Vec
* Highway2Vec

### Simple embeddings

In [ ]:
from srai.loaders import OSMPbfLoader
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER

area = geocode_to_region_gdf(area_name)
features = OSMPbfLoader().load(area, HEX2VEC_FILTER)
regions = H3Regionalizer(resolution=9).transform(area)

In [ ]:
from srai.joiners import IntersectionJoiner

joint = IntersectionJoiner().transform(regions, features)

In [ ]:
from srai.embedders import CountEmbedder

embedder = CountEmbedder()
embeddings = embedder.transform(regions, features, joint)

embeddings.head(3)

In [ ]:
from srai.plotting import plot_regions, plot_numeric_data

folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.4)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, "amenity_bicycle_parking", embeddings, map=folium_map)

## 🏗️ What are we working on?

* Pre-calculated embeddings
* Fine-tuning
* Datasets and benchmarks

## Questions?